<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-обзор-данных" data-toc-modified-id="Загрузка-и-обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и обзор данных</a></span><ul class="toc-item"><li><span><a href="#Импорты-и-настройки" data-toc-modified-id="Импорты-и-настройки-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорты и настройки</a></span></li><li><span><a href="#Загрузка" data-toc-modified-id="Загрузка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка</a></span></li><li><span><a href="#Обзор" data-toc-modified-id="Обзор-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Обзор</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Проверка-корректности-дешифровки-данных" data-toc-modified-id="Проверка-корректности-дешифровки-данных-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка корректности дешифровки данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка и обзор данных

### Импорты и настройки

In [1]:
import warnings

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Сделаем удобные настройки вывода.

In [2]:
# Зададим формат отображения вещественных чисел в Pandas
pd.set_option('display.float_format', '{:,.2f}'.format)

# Сбросим ограничение на число столбцов
pd.set_option('display.max_columns', None)

# Сбросим ограничение на число строк
pd.set_option('display.max_rows', None)

# Не показываем предупреждения
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# Зададим константу для генератора псевдослучайных чисел
RANDOM_STATE = 42

# Зададим значение для генератора псевдослучайных чисел
np.random.seed(RANDOM_STATE)

### Загрузка

In [3]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('insurance.csv')

### Обзор

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Колонка Возраст имее тип данных float, это странно. Выведем случайные пять строк датафрейма.

In [5]:
df.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
1501,1,28.00,"56,100.00",0,0
2586,1,32.00,"41,900.00",2,0
2653,1,30.00,"26,300.00",0,0
1055,1,30.00,"37,300.00",2,0
705,0,44.00,"30,000.00",1,1


In [6]:
# Посмотрим на все уникальные значения колонки Возраст
df['Возраст'].value_counts()

19.00    223
25.00    214
31.00    212
26.00    211
27.00    209
22.00    209
32.00    206
28.00    204
29.00    203
30.00    202
23.00    202
21.00    200
20.00    195
36.00    193
33.00    191
24.00    182
35.00    179
34.00    177
37.00    147
39.00    141
38.00    139
41.00    129
18.00    117
40.00    114
42.00     93
43.00     77
44.00     74
45.00     73
46.00     60
47.00     47
49.00     37
50.00     27
48.00     26
52.00     22
51.00     21
53.00     11
55.00      9
54.00      7
56.00      5
59.00      3
60.00      2
58.00      2
57.00      2
65.00      1
61.00      1
62.00      1
Name: Возраст, dtype: int64

Напишем функцию для обзора данных.

In [7]:
def dataframe_information(data):
    """Функция для вывода основных статистик набора данных"""
    df_data = []
    df_cols = ['name', 'object', 'na', 'zero', 'rate', 'unique', 'neg',
               'mean', 'std', 'min', 'max', 'outliners', 'duplicates']
    # перебираем столбцы в наборе данных
    for column_name in data.columns:
        column_negative_values, column_mean, column_std, column_min, column_max, outliers, duplicates_sum = \
            None, None, None, None, None, None, None
        # считаем характеристики
        column_type = data[column_name].dtypes
        column_na_values = data[column_name].isna().sum()
        column_zero_values = data[column_name][data[column_name] == 0].count()
        column_na_zero_rate = ((column_na_values + column_zero_values) / data.shape[0]) * 100
        column_unique_values = len(data[column_name].unique())
        duplicates_sum = len(data.loc[data.duplicated() > 0, column_name])
        if data[column_name].dtype != 'object':
            column_negative_values = data[column_name][data[column_name] < 0].count()
            column_mean = data[column_name].mean()
            column_std = data[column_name].std()
            column_min = data[column_name].min()
            column_max = data[column_name].max()
            q1 = data[column_name].quantile(0.25)
            q3 = data[column_name].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            outliers = len(data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)])

        # собираем показатели по каждому полю
        df_data.append([column_name, column_type, column_na_values,
                        column_zero_values, column_na_zero_rate, column_unique_values,
                        column_negative_values, column_mean, column_std,
                        column_min, column_max, outliers, duplicates_sum])

    # формируем набор данных
    df_res = pd.DataFrame(data=df_data, columns=df_cols)

    return df_res


# Применим функцию на датафрейм
dataframe_information(df)

,name,object,na,zero,rate,unique,neg,mean,std,min,max,outliners,duplicates
0,Пол,int64,0,2505,50.10,2,0,0.50,0.50,0.00,1.00,0,153
1,Возраст,float64,0,0,0.00,46,0,30.95,8.44,18.00,65.00,12,153
2,Зарплата,float64,0,0,0.00,524,0,"39,916.36","9,900.08","5,300.00","79,000.00",37,153
3,Члены семьи,int64,0,1513,30.26,7,0,1.19,1.09,0.00,6.00,7,153
4,Страховые выплаты,int64,0,4436,88.72,6,0,0.15,0.46,0.00,5.00,564,153


Обнаружены следующие особенности: \
- В данных есть дубликаты - 153.
- Колонки Возраст и Зарплата имеют тип данных float, здесь этот тип избыточен, приведем его к типу данных int.

Таким образом данные имеют 5000 тыс случаев и 5 признаков, из которых один категориальный (пол), остальные количественные. \
Целевой признак - колонка с количеством страховых выплат.

In [8]:
# Удалим дубликаты
df = df.drop_duplicates()

In [9]:
# Изменим тип данных на int
df[['Возраст', 'Зарплата']] = df[['Возраст', 'Зарплата']].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


## Умножение матриц

Уточним, изменится ли качество линейной регрессии, если умножать признаки на обратимую матрицу.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — обратная матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Нам нужно доказать, что при уножении на обратимую матрицу, предсказание не изменится:

$$
a = Xw = XPw'$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y$$

Тогда:

$$
w' = (X^T X)^{-1} X^T y = ((XP)^T (XP))^{-1} (XP)^T y$$


Согласно теории, квадратная матрица является обратимой, если существует матрица B, отвечающая условиям:
$$
AB = BA = E$$
где $E$ - единичная матрица.

Если матрица A обратима, то обратная к ней обозначается как $A^{-1}$ и условия выше можно записать как:
$$
AA^{-1} = A^{-1}A = E$$

Также нам понадобятся следующие свойства матриц:
$$
AE = EA = A$$ 
$$
(AB)^{-1} = B^{-1} A^{-1}$$
$$
(AB)^T = B^T A^T$$

Используя эти свойства, преобразуем выражение для w':

$$
w' = ((XP)^T (XP))^{-1} (XP)^T y$$
$$   = (P^T X^T X P)^{-1} P^T X^T y$$
$$   = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y$$
$$   = P^{-1} (X^T X)^{-1} X^T y$$

Подытожим:

$$
a = X (X^T X)^{-1} X^T y = X P P^{-1} (X^T X)^{-1} X^T y$$
$$
a = X (X^T X)^{-1} X^T y = X (X^T X)^{-1} X^T y$$

Формулы одинаковые.

Таким образом, мы можем сделать вывод, что предсказания модели не изменятся при использовании обратимой матрицы $P$ для преобразования признаков.

Согласно теории, квадратная матрица является обратимой, если существует матрица B, отвечающая условиям:
$$
AB = BA = E$$
где $E$ - единичная матрица.

Если матрица A обратима, то обратная к ней обозначается как $A^{-1}$ и условия выше можно записать как:
$$
AA^{-1} = A^{-1}A = E$$

Также нам понадобятся следующие свойства матриц:
$$
AE = EA = A$$ 
$$
(AB)^{-1} = B^{-1} A^{-1}$$
$$
(AB)^T = B^T A^T$$


Приступим к расчетам.

Формула минимизации потерь для задачи обучения имеет вид:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула расчета весов:

$$
w = (X^T X)^{-1} X^T y$$

Домножаем матрицу признаков на обратимую матрицу, тогда формулы будет иметь вид:

$$
w_p = \arg\min_w MSE(XPw_p, y)
$$

$$
w_p = ((XP)^T (XP))^{-1} (XP)^T y$$

Расскроем скобки:

$$
w_p = ((XP)^T (XP))^{-1} (XP)^T y$$
$$  =  (P^T X^T XP)^{-1} P^T X^T y$$
$$   = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y$$
$$   = P^{-1} (X^T X)^{-1} X^T y$$
$$   = P^{-1} w$$

Это означает, что параметры линейной регрессии в преобразованной задаче могут быть получены путем умножения параметров в исходной задаче на обратную матрицу $P^{-1}$.

Подставим в формулу обучения:
$$
w_p = \arg\min_w MSE(XPP^{-1} w, y)$$

Так как $PP^{-1} = E$, а умножение на единичную матрицу не изменяет исходную, все сводится к:

$$
\arg\min_w MSE(X w, y)$$

Таким образом, мы можем сделать вывод, что предсказания модели не изменятся при использовании обратимой матрицы $P$ для преобразования признаков.

## Алгоритм преобразования

Предложим алгоритм преобразования данных для решения задачи. Обоснуем, почему качество линейной регрессии не поменяется.

1. Отделим таргет, разделим выборки на обучающие и тестовые.
2. Получим случайную квадратную матрицу размера 4 х 4 (сторона равна количеству признаков, за минусом таргета), воспользуемся функцией np.random.normal, чтобы повысить шансы получить обратимую матрицу.
3. Проверим, что матрица обратима, если же она является вырожденной или сингулярной вернемся к пункту 2. 
4. Шифруя признаки умножим исходную матрицу на обратимую, оформим результат в датафрейм.
5. Обучим моделии линейной регрессии на каждом датафрейме, получим предсказания.
6. Сравним их метрикой R2 (коэффициент детерминации).

Метрика R2 считается так:
$$
R2 = 1 - \frac{MSE} {D_0}
$$

где $D_0$ это дисперсия таргета в тестовых фреймах. \
Мы воспрользуемся готовой функцией r2_score из библиотеки sklearn.

Качество линейной регрессии не поменяется, потому что умножение на обратимую матрицу только равномерно смещает в пространстве вектор признаков, но не изменяет его направление, поэтому связи между признаками и целевой переменной остаются теми же, и модель линейной регрессии продолжает использовать ту же информацию для предсказания. \
А благодаря тому, что мы умножаем признаки на **обратимую** матрицу, мы можем восстановить исходные данные умножив зашифрованную матрицу на обратную матрицу этой обратимой матрицы, так как $AA^{-1} = E$, а умножение на единичную матрицу не изменяет исходную.

## Проверка алгоритма

In [10]:
# Отделим таргет
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

# Разделим на обучающую и тестовые выбоки
features_train, features_test, target_train, target_test = (
    train_test_split(features, target, test_size=0.25, random_state=RANDOM_STATE))

Для получения обратимой матрицы напишем функцию. Добавим рекурсивный вызов в случае если матрица не обратима.

In [11]:
def get_new_matrix(size: tuple):
    new_matrix = np.random.normal(size=size)
    determinant = np.linalg.det(new_matrix)
    if determinant != 0:
        return new_matrix
    else:
        get_new_matrix(size)
        
new_matrix = get_new_matrix((features.shape[1], features.shape[1]))

new_matrix

array([[ 1.91240941,  0.86415421,  0.27861919, -1.42733359],
       [ 0.20202094,  0.21494017, -0.19864026,  0.63296765],
       [ 1.12190571, -0.81840393, -0.83398435, -0.13929201],
       [-0.61522919, -1.6412532 , -0.99659274, -1.23232115]])

In [12]:
# Перемножим матрицы
new_features = features.values @ new_matrix

# Оформим в датафрейм
new_features = pd.DataFrame(new_features, columns=features.columns)

new_features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,"55,656.10","-40,584.80","-41,374.49","-6,885.59"
1,"42,641.09","-31,091.10","-31,701.54","-5,265.21"
2,"23,565.88","-17,180.25","-17,519.43","-2,906.78"
3,"46,786.48","-34,126.21","-34,783.31","-5,797.65"
4,"29,289.31","-21,353.46","-21,772.27","-3,619.23"


In [13]:
# Разделим на обучающую и тестовую выбоки
new_features_train, new_features_test = (
    train_test_split(new_features, test_size=0.25, random_state=RANDOM_STATE))

Обучим модель линейной регрессии, получим предсказания на незашифрованных данных и измерим коэффициент детерминации.

In [14]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(f'Метрика R2 по исходным данным {r2_score(target_test, predictions):.5f}')

Метрика R2 по исходным данным 0.44346


На зашифрованных:

In [15]:
model = LinearRegression()
model.fit(new_features_train, target_train)
new_predictions = model.predict(new_features_test)
print(f'Метрика R2 по зашифрованным данным {r2_score(target_test, new_predictions):.5f}')

Метрика R2 по зашифрованным данным 0.44346


Метрики не отличаются, поэтому данный метод можно использовать для шифрования данных клиентов, это не повлияет на качество предсказаний модели линейной регрессии. 

## Проверка корректности дешифровки данных

In [33]:
# Вычислим обратную матрицу
inverse_matrix = np.linalg.inv(new_matrix)
inverse_matrix

array([[-2.43165521e-03, -7.86079042e-01,  7.69469978e-01,
        -4.87918952e-01],
       [ 6.71218641e-01,  2.60223075e+00, -1.22987757e+00,
         6.98184152e-01],
       [-5.92932584e-01, -3.58153606e+00,  9.63708392e-01,
        -1.26178209e+00],
       [-4.13229469e-01, -1.76875940e-01,  4.74482911e-01,
        -4.77336714e-01]])

Дешифруем и оформим в датафрейм результат.

In [39]:
decrypted_matrix = new_features.values @ inverse_matrix

decrypted_matrix = pd.DataFrame(decrypted_matrix, columns=features.columns)

# Выведем на экран дешифрованную матрицу
decrypted_matrix.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1.00,41.00,"49,600.00",1.00
1,0.00,46.00,"38,000.00",1.00
2,-0.00,29.00,"21,000.00",-0.00
3,0.00,21.00,"41,700.00",2.00
4,1.00,28.00,"26,100.00",-0.00


Выведем на экран исходный датафрейм признаков.

In [37]:
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Как видим данные совпадают, дешифровка данных путем умножения на обратную матрицу (обратимой матрицы, использовавшейся для шифрования данных) корректно работает. 

## Вывод

Нам необходимо было разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обосновать корректность его работы. Было важно, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

В ходе работы мы провели обзор данных, при котором нами было выявлено небольшое количество дубликатов (~3%), которые в ходе предобработки мы удалили, так как такое количество все таки слишком большое для случайного совпадения.

В данной работе мы теоретически обосновали, написали алгоритм действий и проверили на практике корректность шифрования данных методом умножения векторов признаков на обратимую матрицу. 
В практической части мы обучили две модели линейной регрессии на исходных и зашифрованных данных, и измерили точность их предсказаний метрикой R2. 
Обе модели показали одинаковый коэффициент детерминации на тестовых данных, поэтому данный метод можно использовать для шифрования данных клиентов. 

Также мы проверили корректность дешифровки данных, путем уножения зашифрованной матрицы на обратную матрицу обратимой матрицы, которую мы использовали для шифрования данных. Дешифровка корректно работает. 

Задачу бизнеса мы решили. 